<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:5px 5px;">
    <i>Aprender una técnica no es un fin en si mismo, simplemente indica por donde hay que empezar</i><br><b>Proverbio Japonés</b></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:right;border-radius:5px 5px;">
<i>Material curado y organizado por el Prof. Dr. Hernán D. Merlino</i><br> 
<i>Retrieval Augmented Generation (RAG)  Versión 2.0 - 2024 Q3</i></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>R</strong>etrieval <strong>A</strong>ugmented <strong>G</strong>eneration</p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Cargar librerías</strong></p> 

In [8]:
import os
BASE_DIR = '../'
PATH_TO_TRAIN = 'productos.csv'

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
from transformers import pipeline

In [9]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv(PATH_TO_TRAIN, sep=';')

# Filtrar columnas relevantes
df = df[['nme_product', 'nme_target', 'nme_crop']]

# Eliminar duplicados y datos faltantes
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# Verificar el conjunto de datos procesado
print(df.head())

          nme_product             nme_target nme_crop
0  7344 VT Triple PRO        Isoca bolillera     Maíz
1  7344 VT Triple PRO  Oruga militar tardía      Maíz
2  7349 VT Triple PRO        Isoca bolillera     Maíz
3  7349 VT Triple PRO  Oruga militar tardía      Maíz
5         Acrobat® MZ           Peronospora       Vid


In [10]:
df['texto'] = df.apply(lambda row: f"El producto {row['nme_product']} combate a la plaga {row['nme_target']} y se usa en el cultivo {row['nme_crop']}.", axis=1)

<p style="font-size:17px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<a href="https://huggingface.co/docs/transformers/main_classes/pipelines" target=\"_blank\">Pipelines Method</a></p>

<p style="font-size:17px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<a href="https://huggingface.co/docs/transformers/v4.46.0/en/add_new_pipeline" target=\"_blank\">How to create a custom pipeline?</a></p>

In [11]:
documents = [f'"{texto}."' for texto in df['texto']]

In [12]:
generator = pipeline(task="text-generation",model="datificate/gpt2-small-spanish")

<p style="font-size:17px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<a href="https://huggingface.co/models?pipeline_tag=text-generation&sort=trending&search=spanish" target=\"_blank\">Spanish models</a></p>

<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<a href="https:///huggingface.co/datificate/gpt2-small-spanish" target=\"_blank\">GPT2-small-spanish is a state-of-the-art language model for Spanish based on the GPT-2 small model</a></p>

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Corpus de documentos</strong></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Vectorización</strong></p> 

In [13]:
documents

['"El producto 7344 VT Triple PRO combate a la plaga Isoca bolillera y se usa en el cultivo Maíz.."',
 '"El producto 7344 VT Triple PRO combate a la plaga Oruga militar tardía  y se usa en el cultivo Maíz.."',
 '"El producto 7349 VT Triple PRO combate a la plaga Isoca bolillera y se usa en el cultivo Maíz.."',
 '"El producto 7349 VT Triple PRO combate a la plaga Oruga militar tardía  y se usa en el cultivo Maíz.."',
 '"El producto Acrobat® MZ combate a la plaga Peronospora  y se usa en el cultivo Vid.."',
 '"El producto Acrobat® MZ combate a la plaga Tizón tardío de la papa y se usa en el cultivo Papa.."',
 '"El producto Acronis® combate a la plaga Mancha ojo de rana y se usa en el cultivo Soja.."',
 '"El producto Acronis® combate a la plaga Tizón de la vaina y se usa en el cultivo Algodón.."',
 '"El producto Acronis® combate a la plaga Botrytis y se usa en el cultivo Papa.."',
 '"El producto Acronis® combate a la plaga Mancha de tallo y vaina y se usa en el cultivo Arroz.."',
 '"El pr

In [14]:
vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(documents)

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Métodos</strong></p> 

In [15]:
def retrieve_document(query, vectorizer, doc_vectors):
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, doc_vectors)
    most_similar_idx = similarities.argmax()
    return documents[most_similar_idx]

In [29]:
def rag_example(query):
    document = retrieve_document(query, vectorizer, doc_vectors)
    answer = generator(document,
                       max_length=50,
                       num_return_sequences=1,
                       temperature=0.95,
                       top_k=50,  
                       top_p=0.9)
    return answer[0]["generated_text"]

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong><u>Params</u></strong>:
<ul style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<li><i><b>document:</b></i> texto base</li>
<li><i><b>max_length:</b></i> genera hasta 50 tokens</li>
<li><i><b>num_return_sequences:</b></i> retorna solo un texto</li>
<li><i><b>temperature:</b></i> ajusta la creatividad del modelo</li>
<li><i><b>top_k y top_p:</b></i> controlan la selección de palabras para una generación más coherente y variada</li>
</ul></p>

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Pregunta</strong></p> 

In [66]:
query = "Tengo un problema con el sorgo de alepo en mi cultivo de maíz"
answer = rag_example(query)

In [67]:
answer_cortada = answer.split('.')[0]

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px 5px;">
<strong>Respuesta</strong></p> 

In [68]:
print(answer_cortada)

"El producto Liberty® combate a la plaga Sorgo de Alepo de rizoma  y se usa en el cultivo Maíz


<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:15px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px 5px;">
<strong>_____________________________________________________________</strong></p> 